# Notes on Residual Pooling

Notes describing the residual pooling algorithm for time-series compression.

## Definitions
Let's assume that we have a univariate time-series represented as a vector of data.
$$\mathbf{x} = [x_0,x_1,...,x_T], x_i \in \mathbb{R}, x \in \mathbb{R}^T$$

### Pooling
**Pooling** is an operation that reduces the dimensionality of a data series in a specific way. The pooling operation is defined by two parameters a kernel width $w$ and function $f$ that maps slices of the time-series of size $k$ to a scalar value.
$$\rho_{f,w}(\mathbf{x}) = \mathbf{p}[j] = f([x_{w\cdot(j-1)},...,x_{w\cdot(j)}])$$

For this to work without ambiguity $T$ must be divisible by $k$. Let's see some examples of different pooling operations. Let's consider the "mean" pool operation over a kernel width of size $2$. In math this can be expressed as follows:
$$\rho_{mean,2}(\mathbf{x}) = \mathbf{p}[j] = mean([x_{w\cdot(j-1)},...,x_{w\cdot(j)}])$$

and in code, this is expressed as:

In [15]:
import numpy as np

def mean_pool_2(x):
    return np.mean(x.reshape(-1,2), axis=1)

print(mean_pool_2(np.array([1,3,4,0,6,7])))
print(mean_pool_2(np.array([1,3,1,2])))

[2.  2.  6.5]
[2.  1.5]


The operation takes each window of size 2 and calculates the mean value. One could imagine a similar operation that takes the median:

In [12]:
def median_pool_3(x):
    return np.median(x.reshape(-1,3),axis=1)

print(median_pool_3(np.array([1,3,4,0,6,7])))

[3. 6.]


Even "stratified" sampling can be expressed as a pooling operation. Consider the function that takes a window of size 3 and extracts a single random element from each:

In [14]:
def sample_pool_3(x):
    slices = x.reshape(-1,3)
    N,_ = slices.shape
    
    return np.array([np.random.choice(slices[i]) for i in range(N)])

print(sample_pool_3(np.array([1,3,4,0,6,7])))

[1 7]


We can write a generic pooling function as follows

In [19]:
def pool(x, fn, width):
    slices = x.reshape(-1,width)
    N,_ = slices.shape
    
    return np.array([fn(slices[i]) for i in range(N)])

print(pool(np.array([1,3,4,0,6,7]), np.mean, 2))
print(pool(np.array([1,3,4,0,6,7]), np.max, 2))
print(pool(np.array([1,3,4,0,6,7]), np.median, 6))

[2.  2.  6.5]
[3 4 7]
[3.5]


### Spline Interpolation

Pooling is usually a lossy operation since it encodes a slice of size $w$ with a single number. Thus, inverting this operation will only give us an approximation of the original series. 

We have some function $s$ that tries to estimate the original time-series from the pooled values.
$$ s(p_{f,w}(\mathbf{x})) \approx \mathbf{x} $$

The simplest way to do this is to duplicate each value $w$ times:
$$ s(\mathbf{p}) = [\mathbf{p}[0],\mathbf{p}[0],...\times w,...,\mathbf{p}[T/w],\mathbf{p}[T/w]]$$

This is a specical case of spline interpolation. We can think of each function of a slice as a summary of the values in that slice. The problem of inversion is similar to an interpolation problem where each slice can be thought as a sample of the original data. Spline interpolation is a form of interpolation where the interpolant is a special type of piecewise polynomial called a spline. That is, instead of fitting a single, high-degree polynomial to all of the values at once, spline interpolation fits low-degree polynomials to small subsets of the values. In the duplication case above one is simply fitting a constant function.

We model the interpolation in a simple way where every $\mathbf{p}[j]$ can be thought of as an estimate for the midpoint of the interval it represents. Let's see some examples of how this works with different interpolation techniques from `scipy.interpolate.interp1d`.

In [99]:
from scipy.interpolate import interp1d

def spline(p, width, inter):
    N = p.shape[0]
    
    #degenerate case
    if N == 1:
        return np.ones(N*width)*p[0]
    
    #treat every obs as midpoint of its range
    fn = interp1d(np.arange(0,N*width, width) + (width-1)/2, p, \
                  kind=inter, fill_value="extrapolate")
    
    return fn(np.arange(0,N*width,1))

In the simplest case, this function can return the duplication interpolation that we discussed before:

In [100]:
spline(np.array([0,1,2]),2,'nearest')

array([0., 0., 1., 1., 2., 2.])

This function can act as a reasonable inverse for pooling functions if the original time-series is very consistent in terms of its neighboring values.

In [101]:
x = np.array([1.04,1.1,1.36,1.4,1.52,1.7])
p = pool(x, np.mean, 2)
xh = spline(p, 2, 'nearest')

print('Data: ', x)
print('Mean2 Pool: ', p)
print('Reconstruction: ', xh)
print('Errors: ', x-xh)

Data:  [1.04 1.1  1.36 1.4  1.52 1.7 ]
Mean2 Pool:  [1.07 1.38 1.61]
Reconstruction:  [1.07 1.07 1.38 1.38 1.61 1.61]
Errors:  [-0.03  0.03 -0.02  0.02 -0.09  0.09]


However, this might not be the best approach in all cases. Consider the following data, where there is a consistent trend:

In [102]:
x = np.array([0,1,2,3,4,5])
p = pool(x, np.mean, 2)
xh = spline(p, 2, 'nearest')

print('Data: ', x)
print('Mean2 Pool: ', p)
print('Reconstruction: ', xh)
print('Errors: ', x-xh)

Data:  [0 1 2 3 4 5]
Mean2 Pool:  [0.5 2.5 4.5]
Reconstruction:  [0.5 0.5 2.5 2.5 4.5 4.5]
Errors:  [-0.5  0.5 -0.5  0.5 -0.5  0.5]


The first value of the reconstruction of each slice systematically under-estimates and the second over-estimates. We can fix this by using a higher-order interpolation. Consider interpolating the data with a linear function.

In [103]:
x = np.array([0,1,2,3,4,5])
p = pool(x, np.mean, 2)
xh = spline(p, 2, 'linear')

print('Data: ', x)
print('Mean2 Pool: ', p)
print('Reconstruction: ', xh)
print('Errors: ', x-xh)

Data:  [0 1 2 3 4 5]
Mean2 Pool:  [0.5 2.5 4.5]
Reconstruction:  [0. 1. 2. 3. 4. 5.]
Errors:  [0. 0. 0. 0. 0. 0.]


## Residual Vectors
The error vectors in both of the cases above are really interesting, because while the time-series contain six distinct valuees, the error vectors are more repetitive. We call these vectors the "residual" vectors. 

$$ \mathbf{r} = \mathbf{x} - s(\mathbf{p})$$

The key insight of our work is that residual vectors are generally more compressible than the original time-series. This is the same basic insight in delta coding but we will show how we can extend it. 

In [135]:
def residual(x, pool_fn, spline_fn):
    p = pool_fn(x)
    xh = spline_fn(p)
    return xh, x - xh

x = np.array([0,1,2,3,4,5])
residual(x, lambda y: pool(y, np.mean, 2), \
            lambda y: spline(y,2, 'nearest'))

(array([0.5, 0.5, 2.5, 2.5, 4.5, 4.5]),
 array([-0.5,  0.5, -0.5,  0.5, -0.5,  0.5]))

It is clearly true that $\mathbf{x} = s(\mathbf{p}) + \mathbf{r}$, simply by definition: 

In [105]:
x = np.array([0,1,2,3,4,5])
xh, r = residual(x, lambda y: pool(y, np.mean, 2), \
            lambda y: spline(y,2, 'nearest'))
print(x, xh+r)

[0 1 2 3 4 5] [0. 1. 2. 3. 4. 5.]


Now, let's further consider *applying a pooling operation to* $\mathbf{r}$ independent of $\mathbf{x}$. This would result in a $\mathbf{p}_r$ and a new residual $\mathbf{r}_2$. The same equality above would hold:
$$\mathbf{r} = \mathbf{r}_2 + s(\mathbf{p}_r) $$

Since the residuals are additive, it is easy to see that:
$$\mathbf{x} = s(\mathbf{p}) + s(\mathbf{p}_r) + \mathbf{r}_2$$

We can repeat this operation any number of times pooling the resulting residual vector.
Suppose, we constructed a sequence of pooling and spline operations in such a way that the final $\mathbf{r}_i$ was guaranteed to be 0, then it is sufficient to store each of the pooled residuals to fully reconstruct $\mathbf{x}$.

One way to achieve this goal is to create a heirarchy of pooling functions. Let's assume that $T = 2^k$, i.e., $T$ is a power of two. Consider the following sequence of pooling and interpolation:
$$\mathbf{r_0} = \mathbf{x}$$
$$\mathbf{p_0} = p_{f,2^0}(\mathbf{r_0})$$
$$\mathbf{r_1} = \mathbf{r_0} - s(\mathbf{p_0})$$
$$\mathbf{p_1} = p_{f,2^1}(\mathbf{r_1})$$
$$\mathbf{r_2} = \mathbf{r_1} - s(\mathbf{p_1})$$
$$...$$
$$\mathbf{p_{k-1}} = p_{f,2^{k-1}}(\mathbf{r_{k-1}})$$
$$\mathbf{r_k} = \mathbf{r_{k-1}} - s(\mathbf{p_{k-1}})$$

With this hierarchy, we can show that:
$$\mathbf{x} = \sum_{i=0}^k s(\mathbf{p_0}) $$

Let's see this in code to better understand how it works.

In [113]:
def hierarchy(x):
    curr = x.copy().astype(np.float64) #copy x
    N = x.shape[0]
    k = int(np.log2(N))+1
    
    pooled = []
    
    for i in range(k):
        w = N // (2**i)
        v = pool(curr, np.mean, w)
        curr -= spline(v, w, 'nearest')
        
        pooled.append(v)
    
    return pooled

hierarchy(np.array([1,2,4,5,6,7,10,11]))

[array([5.75]),
 array([-2.75,  2.75]),
 array([-1.5,  1.5, -2. ,  2. ]),
 array([-0.5,  0.5, -0.5,  0.5, -0.5,  0.5, -0.5,  0.5])]

To reconstruct the original array, we simply have to do the following:

In [152]:
def reconstruct(hier, sfn='nearest'):
    N = hier[-1].shape[0]
    x = np.zeros(N)

    for arr in hier:
        w = N//arr.shape[0]
        x += spline(arr, w, sfn)
        
    return x

h = hierarchy(np.array([1,2,4,5,6,7,10,11]))
reconstruct(h)

array([ 1.,  2.,  4.,  5.,  6.,  7., 10., 11.])

**What I suspect is true** For any consistent pooling function (one that is lossless with $w=1$) and interpolating spline (one that is lossless at the control points), this hierarchy is always lossless.

## Lossy Residual Vectors
Suppose, we now were fine with a lossy representation of the data. That is for reconstruction we were willing to tolerate an error on each element of at most $\epsilon$. This is realy to engineer, we just have to zero out all residuals that have an absolute value less than that.

In [162]:
def hierarchy_lossy(x, epsilon=0, sfn='nearest', pfn=np.mean):
    curr = x.copy().astype(np.float64) #copy x
    N = x.shape[0]
    k = int(np.log2(N))+1
    
    pooled = []
    
    for i in range(k):
        w = N // (2**i)
        v = pool(curr, pfn, w)
        curr -= spline(v, w, sfn)
        
        
        #new code
        _, r = residual(curr, lambda y: pool(y, pfn, w), \
                           lambda y: spline(y, w, sfn))
        #step 1. calculate residual
        
        rp = pool(np.abs(r), np.max, w)
        #step 2. find max abs error in each pool
        
        mask = np.repeat((rp <= epsilon), w).astype(np.bool)
        #step 3. generate a zero'ing out mask for the residual
    
        curr[mask] = 0
        #Step 4. zero out residual
        
        if i == k - 1:
            v[np.abs(v) <= epsilon] = 0
        #Step 5. zero out last stored pooled residual (can also quantize)
        
        pooled.append(v)
    
    return pooled

If we set epsilon to zero, then we have a lossless hierarchy

In [163]:
print(hierarchy_lossy(np.array([1,2,4,5,6,7,10,11]), epsilon=0))

[array([5.75]), array([-2.75,  2.75]), array([-1.5,  1.5, -2. ,  2. ]), array([-0.5,  0.5, -0.5,  0.5, -0.5,  0.5, -0.5,  0.5])]


<ipython-input-162-30625ad19d42>:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.repeat((rp <= epsilon), w).astype(np.bool)


Let's set $\epsilon$ to 0.5, and then see what happens:

In [164]:
compressed = hierarchy_lossy(np.array([1,2,4,5,6,7,10,11]), epsilon=0.5)
print('Compressed', compressed)

Compressed [array([5.75]), array([-2.75,  2.75]), array([-1.5,  1.5, -2. ,  2. ]), array([0., 0., 0., 0., 0., 0., 0., 0.])]


<ipython-input-162-30625ad19d42>:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.repeat((rp <= epsilon), w).astype(np.bool)


The entire last level of the hierarchy is all zero and doesn't have to be stored.

In [165]:
print('Reconstruction', reconstruct(compressed))
print('Original', np.array([1,2,4,5,6,7,10,11]))

Reconstruction [ 1.5  1.5  4.5  4.5  6.5  6.5 10.5 10.5]
Original [ 1  2  4  5  6  7 10 11]


We can try out different pooling and spline functions as well. These result in more zeros in the representation that can be easily compressed.

In [166]:
compressed = hierarchy_lossy(np.array([1,2,4,5,6,7,10,11]), epsilon=0.5, sfn='linear')
print('Compressed', compressed)
print('Reconstruction', reconstruct(compressed, sfn='linear'))
print('Original', np.array([1,2,4,5,6,7,10,11]))

Compressed [array([5.75]), array([-2.75,  2.75]), array([ 0.   ,  0.   , -0.625,  0.625]), array([0., 0., 0., 0., 0., 0., 0., 0.])]
Reconstruction [ 0.9375   2.3125   3.6875   4.90625  5.96875  7.5      9.5     11.5    ]
Original [ 1  2  4  5  6  7 10 11]


<ipython-input-162-30625ad19d42>:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.repeat((rp <= epsilon), w).astype(np.bool)


In [167]:
compressed = hierarchy_lossy(np.array([1,2,4,5,6,7,10,11]), epsilon=0.5, pfn = np.median,sfn='linear')
print('Compressed', compressed)
print('Reconstruction', reconstruct(compressed, sfn='linear'))
print('Original', np.array([1,2,4,5,6,7,10,11]))

Compressed [array([5.5]), array([-2.5,  3. ]), array([ 0.   ,  0.   , -0.625,  0.625]), array([0., 0., 0., 0., 0., 0., 0., 0.])]
Reconstruction [ 0.9375   2.3125   3.6875   4.90625  5.96875  7.5      9.5     11.5    ]
Original [ 1  2  4  5  6  7 10 11]


<ipython-input-162-30625ad19d42>:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.repeat((rp <= epsilon), w).astype(np.bool)
